#### Agent Finetuning
This notebook explores the idea of fientuning language models to behave like agents, for starters we will be considering the use of the fireact finetuning architecture, FireAct focuses on finetuning LLMs to behave like agents following the ReACT framework and for multiple tasks like HotpotQA, StrategyQA and bamboogle.

##### Finetuning an openai model
In this section we will cover finetuning an openai model, particularly GPT3.5. We will be using trajectories generated from gpt-4 to finetune our model, and we will be performing all of this using the openai finetuning API


In [1]:
import os
import json
import re
import openai
import pandas as pd

##### OpenAI files
OpenAI has this concept of files, the files contain the training data which would be used in finetuning our model, we first upload the data as an openai file object, then we can then start a finetuning operation using the uploaded file as our tuning data


In [ ]:
# create openai client
client = openai.OpenAI()

In [ ]:
# show all the files available in your account
files = client.files.list()

In [ ]:
# create dataframe with all the file data sorted by when it was created
pd.DataFrame(sorted(files.data, key=lambda k: -k['created_at']))

In [ ]:
# list all finetuning jobs
jobs = client.fine_tuning.jobs.list()

In [ ]:
# list all the base models and finetuned models this account has access to
models = client.models.list()

In [ ]:
# create new file to be uploaded to openai
file = open("data.jsonl", "r+")

response = client.files.create(file, purpose='fine-tune', user_provided_filename=file.name)

In [ ]:
# create finetuning job
# file_id = response.file_id
file_id = None
n_epochs = 20
job = client.fine_tuning.jobs.create(model="gpt-3.5-turbo", training_file=file_id, hyperparameters={"n_epochs": n_epochs})

In [ ]:
# retreive job
job = client.fine_tuning.jobs.retrieve(job.id)

In [ ]:
# show all the events for a particular job
events = client.fine_tuning.jobs.list_events(job.id)

In [ ]:
# we can also cancel a job 
cancelled_job = client.fine_tuning.jobs.cancel(job.id)

##### Finetuning with Llama2
In this section we will explore finetuning the LLama2 model. We will explore both full finetuning and LoRa finetuning

In [3]:
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence, Union

import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer

In [4]:
StrOrOpenAIObject = Union[str]

In [5]:
@dataclass
class OpenaiDecodingArguments(object):
    max_tokens: int = 1800
    temperature: float = 0.2
    top_p: float = 1.0
    n: int = 1
    stream: bool = False
    stop: Optional[Sequence[str]] = None
    presence_penalty: float = 0.0
    frequency_penalty: float = 0.0
    suffix: Optional[str] = None
    logprobs: Optional[int] = None
    echo: bool = False

In [ ]:
def openai_completion(
    prompts: Union[str, Sequence[str], dict[str, str], Sequence[dict[str, str]]], 
    decoding_args: OpenaiDecodingArguments
):
    pass

In [6]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"


In [7]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [8]:
@dataclass
class ModelArguments:
    model_name_and_path: Optional[str] = field(default="facebook/opt-125m")

In [10]:
@dataclass
class DataArguments:
    data_path: str = field(default=None, metadata={"help": "the path to the training data"})

In [12]:
@dataclass
class TrainingArguments:
    cache_dir: Optional[str] = field(default=None)
    optim: str = field(default="adamw_torch")
    model_max_length: str = field(default=512, 
                                  metadata={"help": "the maximum sequence length, sequences will be right padded or trauncated"})

In [13]:
def safe_save_model_for_hf_trainer(trainer: transformers.Trainer, output_dir: str):
    """ 
    Collects the state dict and saves it on disk
    """
    state_dict = trainer.model.state_dict()
    if trainer.args.should_save == True:
        cpu_state_dict = {key: value.cpu() for key, value in state_dict.items()}
        del state_dict
        trainer._save(output_dir, state_dict=cpu_state_dict)

In [15]:
def smart_tokenizer_and_embedding_resize(special_tokens_dict: dict,
                                         tokenizer: transformers.PreTrainedTokenizer,
                                         model: transformers.PreTrainedModel):
    """ 
    Resize tokenizer and embeddings
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))
    
    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data
        
        input_embeddings_avg = input_embeddings[: -num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[: -num_new_tokens].mean(dim=0, keepdim=True)
        
        
        input_embeddings[-num_new_tokens: ] = input_embeddings_avg
        output_embeddings[-num_new_tokens: ] = output_embeddings_avg
        
    

##### Implement trainer for full model finetuning for Llama2 model
In this section we will be writing the trainer function for full model finetuning for llama2 model

In [ ]:
def train():
    parser = transformers.HfArgumentParser(ModelArguments, DataArguments, TrainingArguments)
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    
    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=training_args.cache_dir
    )
    
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=training_args.cache_dir,
        model_max_length=training_args.model_max_length,
        padding_side="right",
        use_fast=False
    )
    
    if "code" in model_args.model_name_or_path.lower():
        tokenizer.add_eos_token=True
        tokenizer.pad_token_id=0,
        tokenizer.padding_side="left"
        
    if tokenizer.pad_token is None:
        smart_tokenizer_and_embedding_resize(
            special_tokens_dict=dict(pad_token=DEFAULT_PAD_TOKEN),
            tokenizer=tokenizer
            model=model
        )
        
    if "llama" in model_args.model_name_or_path.lower():
        tokenizer.add_special_tokens({
            "eos_token": DEFAULT_EOS_TOKEN,
            "bos_token": DEFAULT_BOS_TOKEN,
            "unk_token": DEFAULT_UNK_TOKEN
        }
        )
    
    
    data_module = make_supervised_data_module()
    trainer = Trainer(model=model, tokenizer=tokenizer, args=training_args, **data_module)
    trainer.train()
    trainer.save_state()
    safe_save_model_for_hf_trainer(trainer, output_dir=training_args.output_dir)